# Introduction to Diffusion MRI

## Diffusion Weighted Imaging (DWI)

Diffusion imaging probes the random, microscopic motion of water protons by employing MRI sequences which are sensitive to the geometry and environmental organization surrounding the water protons. This is a popular technique for studying the white matter of the brain. The diffusion within biological structures, such as the brain, are often restricted due to barriers (eg. cell membranes), resulting in a preferred direction of diffusion (anisotropy). A typical diffusion MRI scan will acquire multiple volumes that are sensitive to a particular diffusion direction and result in diffusion-weighted images (DWI). Diffusion that exhibits directionality in the same direction result in an attenuated signal. With further processing, the acquired images can provide measurements which are related to the microscopic changes and estimate white matter trajectories. Images with no diffusion weighting are also acquired as part of the acquisition protocol.

In [ ]:
dwi = "../../data/sub-01_dwi.nii.gz"
bvec = "../../data/sub-01_dwi.bvec"
bval = "../../data/sub-01_dwi.bval"

In [ ]:
%matplotlib inline

from nilearn import image
from nilearn.plotting import plot_epi

In [ ]:
selected_volumes = image.index_img(dwi, slice(0, 10))

for img in image.iter_img(selected_volumes):
    plot_epi(img, display_mode="z", cut_coords=(30, 53, 75), cmap="gray")

## Diffusion Gradient Scheme

The diffusion-weighted gradient scheme is essential for pre-processing DWI data.

In addition to the acquired diffusion images, two files are collected as part of the diffusion dataset. These files correspond to the gradient amplitude (b-values) and directions (b-vectors) of the diffusion measurement and are named with the extensions `.bval` and `.bvec` respectively. The b-value is the diffusion-sensitizing factor, and reflects the timing & strength of the gradients used to acquire the diffusion-weighted images. The b-vector corresponds to the direction of the diffusion sensitivity. Together these two files define the diffusion MRI measurement as a set of gradient directions and corresponding amplitudes.

In [ ]:
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table

### FSL format

This format consists of a pair of ASCII text files, typically named `.bvec` & `.bval`. The bvals file consists of a single row of space-separated floating-point values, all in one row, with one value per volume in the DWI dataset. The bvecs file consists of 3 rows of space-separated floating-point values, with the first row corresponding to the x-component of the DW gradient vectors, one value per volume in the dataset; the second row corresponding to the y-component, and the third row to the z-component. A typical pair of FSL format DW gradient files might look like:

In the example data below, we see that 2 b-values were chosen for this scanning sequence. The first few images were acquired with a b-value of 0 and are typically referred to as b=0 images. In these images, no DW gradient vector is applied either. These images don't hold any diffusion information and are used as a reference since they aren't subject to the same types of scanner artefacts that affect diffusion-weighted images.  

All of the remaining images have a b-value of 1000 and have a DW gradient vector associated with them. In these images, you can assess the diffusion of water in different directions based on the gradient vector that is applied.

In [ ]:
!cat ../../data/sub-01_dwi.bval

In [ ]:
!cat ../../data/sub-01_dwi.bvec

After reading the `.bval` and `.bvec` files with the `read_bvals_bvecs()` function, we get both in a numpy array. Notice that the `.bvec` file has been transposed so that the x, y, and z-components are in column format.

In [ ]:
gt_bvals, gt_bvecs = read_bvals_bvecs(bval, bvec)

In [ ]:
print("bvals:\n", gt_bvals, "\n\nbvecs:\n", gt_bvecs)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(gt_bvecs.T[0] * gt_bvals, gt_bvecs.T[1] * gt_bvals, gt_bvecs.T[2] * gt_bvals)
plt.show()

It is important to note that in this format, the gradient vectors are provided with respect to the image axes, not in real or scanner coordinates. Simply reformatting the image from sagittal to axial will effectively rotate the b-vectors, since this operation changes the image axes. Thus, a particular bvals/bvecs pair is only valid for the particular image that it corresponds to.

### RASB+ format

Inspired by MRtrix3 and the [BIDS spec](https://github.com/bids-standard/bids-specification/issues/349).

One row per DWI volume, with each row consisting of 4 space-separated floating-point values; these correspond to [x y z b], where [x y z] are the components of the gradient vector, and b is the b-value in units of s/mm2.

In [ ]:
from dmriprep.utils.vectors import DiffusionGradientTable

In [ ]:
?DiffusionGradientTable

In [ ]:
gtab = DiffusionGradientTable(dwi_file=dwi, bvecs=bvec, bvals=bval)

In [ ]:
gtab.gradients

In [ ]:
dwi = "../../data/sub-02_dwi.nii.gz"
bvec = "../../data/sub-02_dwi.bvec"
bval = "../../data/sub-02_dwi.bval"

In [ ]:
gt_bvals, gt_bvecs = read_bvals_bvecs(bval, bvec)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(gt_bvecs.T[0] * gt_bvals, gt_bvecs.T[1] * gt_bvals, gt_bvecs.T[2] * gt_bvals)
plt.show()

The figure below shows an axial view of the brain under different b-values: 0, 1000, and 3000

![bvalue](../images/bvalue.jpg)  
http://mriquestions.com/what-is-the-b-value.html

### Diffusion Spectrum MRI (DSI)

In [ ]:
from dipy.data import get_fnames

In [ ]:
fraw, fbval, fbvec = get_fnames("taiwan_ntu_dsi")

In [ ]:
gt_bvals, gt_bvecs = read_bvals_bvecs(fbval, fbvec)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(gt_bvecs.T[0] * gt_bvals, gt_bvecs.T[1] * gt_bvals, gt_bvecs.T[2] * gt_bvals)
plt.show()